## Experimentação de modelos
 Nesta sessão irei realizar uma série de experimentos, combinando diferentes modelos e preprocessamentos. Para registrar tudo isso, irei utilizar o MLFlow.

Aqui irei testar modelos baseados em distância e árvore, segue os esboços dos testes a serem realizados:

Baseados em distância:
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.


Baseados em árvore:
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas + Isolation Forest para rotular os outliers.
Geral: